# Profitable App Profiles for the App Store and Google Play Markets

This is the first guided project from [Dataquest](dataquest.io). It starts with the case of developers from a company which builds Android and iOS mobile apps. The apps - available on Google Play and App Store - are free to download/install, with revenues made through in-app ads. This implies that the revenue of a given app is largely influenced by the number of its users.

The goal is to understand types of apps that possibly attract more users, using data analysis.

## Initializing Datasets

We start with two datasets:
1. ["AppleStore.csv"](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home), containing ~7K iOS apps in App Store as of June 2017.
2. ["googleplaystore.csv"](https://www.kaggle.com/lava18/google-play-store-apps/home), containing ~10K Android apps in Google Play as of August 2018.

We first open both datasets and convert them to list of lists, separating header and content. The header variables are `app_store_header` and `gogole_play_header`. Meanwhile, the content dataset lists are stored as `app_store` and `google_play`.

In [23]:
import csv

# Open AppleStore.csv file
f1 = open("AppleStore.csv", "r")
app_store_full = list(csv.reader(f1))
app_store_header = app_store_full[0]
app_store = app_store_full[1:]

# Open googleplaystore.csv file
f2 = open("googleplaystore.csv", "r")
google_play_full = list(csv.reader(f2))
google_play_header = google_play_full[0]
google_play = google_play_full[1:]

# Close file handlers for safety
f1.close(); f2.close()

## Exploring Data

First, we introduce a function `explore_data()` which may help to explore our datasets by printing an adjacent couple of rows and finding the number of rows and columns. Below is the code for defining `explore_data()`:

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # Adds a new (empty) line after each row
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

The `explore_data()` takes four inputs: dataset list `dataset`, start integer `start`, end integer `end`, and identity boolean `row_and_columns` (`False` by default). It then slices the dataset from index `start` to index `end`, then prints them with newlines. If `row_and_columns` is `True`, the information of the rows \* columns of the initial dataset is also printed.

We now explore both datasets. Below is the first five rows of `app_store` and `google_play`, setting the parameter `row_and_columns` to `True`:

In [3]:
print("=== First five apps' information in App Store ===", "\n")
    # Adds newline for consistent double spaces with explore_data() function
print("Header: ", app_store_header, "\n")
explore_data(app_store, 0, 5, True)

=== First five apps' information in App Store === 

Header:  ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']

In [4]:
print("=== First five apps' information in Google Play ===", "\n")
print("Header: ", google_play_header, "\n")
explore_data(google_play, 0, 5, True)

=== First five apps' information in Google Play === 

Header:  ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_A

By looking at the headers of each dataset, we can try to identify which column names may contribute to app's attractiveness. As we want to publish free apps, we do not consider price at the moment. Consequently, the columns `'price'` and `'currency'` (in App Store) as well as `'type'` (in Google Play) are excluded.

Based on simplified deductions, here are **several column names that can be further analyzed**, along with their corresponding explanations:

-- For App Store dataset:

1. Average Rating for all versions (`'user_rating'`, column 7)

   Average rating shows how much an app is likeable.
   
2. Number of ratings for all versions (`'rating_count_tot'`, col 5)

   The more ratings, the more stable average rating. 

3. Genre (`'prime_genre'`, col 11)

   Some genre may be more attractive.

4. Size (`'size_bytes'`, col 2)

   Except for very important or impactul apps, lower size is preferred for memory-saving.

5. Number of supporting devices (`'sup_devices.num'`, col 12)

   More supporting devices allows more potential users.

6. Number of screenshots (`'ipadSc_urls.num'`, col 13)

   More screenshots allow people to preview whether an app is visually good, increasing the chance for users to download.

-- For Google Play dataset:
1. Average rating (`'Rating'`, col 2)

2. Number of ratings (`'Reviews'`, col 3)

3. Category/Genre(s) (`'Category'`, col 1; `'Genres'`, col 9)

4. Size (`'Size'`, col 4)

5. Minimum Android Version (`'Android Ver'`, col 12)

   Similar to "number of supporting devices", lower minimum android version allows more potential users.

6. Number of downloads/installs (`'Installs'`, col 5)

   The number of downloads/installs implies popularity.  (some other things can be speculated, but this is just one easy expectation).

The full descriptions of the column names are in [here for App Store data (on the "Content" section)](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) and [here for Google Play data (on the right hand side)](https://www.kaggle.com/lava18/google-play-store-apps#googleplaystore.csv).

## Cleaning Data

The next step is cleaning the data for analysis. We want to detect any inaccurate or duplicate data and act upon them. Inaccurate data can be either corrected or removed, while duplicates can be removed.

We look at [discussion page of Google Play data](https://www.kaggle.com/lava18/google-play-store-apps/discussion) to see if there are any others' comments about these.

### 1. Removing Incorrect Data

Based on [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015), there seems to be an error on the row 10472. Taking into account whether they may have removed the header row, the 10472-nd row of `google_play` reveals the reported error:

In [5]:
print("Header: ", google_play_header, "\n")
print("Row: ", google_play[10472])

Header:  ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Row:  ['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


The row does not have _'Genres'_ entry. Meanwhile, the entries from column _'Category'_ to _'Content Rating'_ seem to mismatch. It is very likely that these entries came from entries from _'Rating'_ to _'Genres'_ which have shifted one to the left.

The _'Category'_ part cannot be filled appropriately, so it may be better to delete this row. We use `del` function, which removes several entries from a list, given the index(s):

In [6]:
del google_play[10472]

### 2. Removing Duplicate Data

Another student reported [here](https://www.kaggle.com/lava18/google-play-store-apps/discussion/76316#latest-448443) that there is an app with duplicate rows but with different number of comments/reviews. From a small number of observations, duplication cases happen when the rows' contents are equal except the _'Reviews'_.

Below is an example of duplications of Instagram app. As you can see, all columnn contents are equal except the _'Reviews'_ column.

In [24]:
print("Header: ", google_play_header, "\n")
for row in google_play:
    app_name = row[0]
    if app_name == "Instagram":
        print("Row: ", row, "\n")

Header:  ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

Row:  ['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 

Row:  ['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 

Row:  ['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 

Row:  ['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 



We assume that the most probable cause of this is because the data is collected multiple times at a different time. Assuming also there is no review cancelation, we can conclude that the "real" row is the one with the most number of reviews, polled at the most recent time.

We now clean duplicate data based on previous principle:

In [8]:
unique_app_reviews_dict = {}
    # A dictionary containing app names as the key
    #     and its maximum number of reviews (among duplicates) as values
for row in google_play:
    app = row[0]
    curr_reviews = row[3]
    if app in unique_app_reviews_dict:
        # The row is a duplicate. Compare which reviews is larger
        unique_app_reviews_dict[app] = max(unique_app_reviews_dict[app], curr_reviews)
    else:
        # The row is the first encountered row with the given app name. Save to dictionary
        unique_app_reviews_dict[app] = curr_reviews
        
# Eliminate the rows in google_play
#     that are not associated with unique_app_reviews_dict
# We introduce two variables:
#     google_play_new that stores the new google_play,
#     and appended_app_list to store app names that are already in google_play_new
google_play_new = []
appended_app_list = []
for row in google_play:
    app = row[0]
    curr_reviews = row[3]
    # If the row has maximum reviews number and is not appended yet in the new dataset,
    #     store the row in the new dataset and "mark" the app name as appended
    if unique_app_reviews_dict[app] == curr_reviews:
        if app not in appended_app_list:
            google_play_new.append(row)
            appended_app_list.append(app)
google_play = google_play_new

# Print dataset length
print("Length: ", len(google_play))

Length:  9659


Now, `google_play` has only 9659 rows, compared to the previous 10841 rows.

### 3. Removing Non English Apps

In this context, the language of the company's apps is still only English, which means that our apps are targeted toward English-speaking markets. Henceforth, removing non-English apps is a good way to reach our goal. We do this heuristically by eliminating apps whose name contains more than three characters not standard in English (i.e. have ASCII number greater than 127).

We first build `isEnglishString()` helper function that checks if a string meets the standard English criteria.

In [9]:
def isEnglishString (string):
    non_eng_char_count = 0
    for char in string:
        if (ord(char) > 127): # ord() gives char's ASCII number
            non_eng_char_count += 1
    return non_eng_char_count <= 3

# Some examples
print(isEnglishString("Docs To Go™ Free Office Suite"))
print(isEnglishString("Docs To Go™™™™ Free Office Suite"))

True
False


We then process with the data cleaning, removing data rows in Google Play whose name does not meet standard English criteria:

In [10]:
print("Previous length: ", len(google_play))
google_play = [row for row in google_play if isEnglishString(row[0])]
print("Current length: ", len(google_play))

Previous length:  9659
Current length:  9614


From above, the number of `google_play` apps has been reduced again from 9659 to 9614.

We can do the same for the other dataset `app_store`. Notice that the app name for this dataset is in the column '`track_name'` (column index: 1).

In [11]:
print("Previous length: ", len(app_store))
app_store = [row for row in app_store if isEnglishString(row[1])]
print("Current length: ", len(app_store))

Previous length:  7197
Current length:  6183


### 4. Isolating Free Apps

Lastly, we would like to remove the paid apps and isolate the free apps, as our company produces free apps. In `google_play` dataset, this is done by comparing the entries in column `'Type'` (col index: 6) with "Free". Meanwhile in `app_store` dataset, the column name is `'price'` (col index: 4, with entries as float numbers with type string) and the compared value is 0.

In [12]:
# google_play dataset

print("Previous length: ", len(google_play))
google_play = [row for row in google_play if row[6] == "Free"]
print("Current length: ", len(google_play))

Previous length:  9614
Current length:  8861


In [13]:
# app_store dataset

print("Previous length: ", len(app_store))
app_store = [row for row in app_store if float(row[4]) == 0]
print("Current length: ", len(app_store))

Previous length:  6183
Current length:  3222


## Analysing Data

### 1. Building App Profiles Based on Genres/Categories

So far we have cleaned our data by removing inaccurate data, duplicates, non-English apps, and non-free apps. Since our apps would be posted in both App Store and Google Play, we are going to pick an app profile that fits both App Store and Google Play.

Our validation strategy is as follow:
1. Build a minimal Android app version, and add it to Google Play.
2. If the app gets good responses, we develop it further.
3. If the app is profitable after six months, we also build its iOS version and add it to App Store.

As described previously in this project (see "Exploring Data"), genres and categories are potentially having some trends in our datasets. **This project will focus on analyzing the attributes of genres and categories**. We will be using frequency tables, with the corresponding columns names are `'Genres'` and `'Category'` for Google Play, and `'prime_genre'` for App Store.

We define two helper functions for creating frequency tables:
1. One for returning a frequency table of its corresponding column of index (in percentage).
2. Other for displaying the top-`n` frequency table for some additional input `n` (`n` is 10 by default).

In [25]:
def freq_table(dataset, index):
    result = {}
    for row in dataset:
        kws = row[index].split(";") # The multiple genres are separated by semicolon
        for kw in kws:
            if kw in result:
                result[kw] += 1
            else:
                result[kw] = 1
    # Convert result value to percentage
    for key in result:
        result[key] *= 100/len(dataset)
    return result

def display_table(dataset, index, top_number = 10):
    # One-liner: looping in a frequency table that has been sorted based on its values
    for key, val in sorted(freq_table(dataset, index).items(), key = lambda x: x[1], reverse = True)[:top_number]:
        print("* ", key, ": ", val, "%")

 We then use the `freq_table()` function above to display the frequency table for the columns `'Genres'` (col idx 9), `'Category'` (col idx 1), and `'prime_genre'` (col idx 11):

In [15]:
print("Frequency tables\n")
print("1. Ten most frequent Genres in Google Play:")
display_table(google_play, 9)
print("\n")
print("2. Ten most frequent Category in Google Play:")
display_table(google_play, 1)
print("\n")
print("3. Ten most frequent prime_genre in App Store:")
display_table(app_store, 11)
print("\n")

Frequency tables

1. Ten most frequent Genres in Google Play:
*  Tools :  8.452770567656021 %
*  Education :  6.838957228303803 %
*  Entertainment :  6.421397133506376 %
*  Business :  4.593161042771697 %
*  Lifestyle :  3.9160365647218147 %
*  Productivity :  3.8934657487868187 %
*  Finance :  3.7016138133393524 %
*  Medical :  3.521047285859384 %
*  Sports :  3.48719106195689 %
*  Personalization :  3.3179099424444196 %


2. Ten most frequent Category in Google Play:
*  FAMILY :  18.92562916149419 %
*  GAME :  9.694165444080804 %
*  TOOLS :  8.452770567656021 %
*  BUSINESS :  4.593161042771697 %
*  LIFESTYLE :  3.9047511567543167 %
*  PRODUCTIVITY :  3.8934657487868187 %
*  FINANCE :  3.7016138133393524 %
*  MEDICAL :  3.521047285859384 %
*  SPORTS :  3.3969077982169056 %
*  PERSONALIZATION :  3.3179099424444196 %


3. Ten most frequent prime_genre in App Store:
*  Games :  58.16263190564866 %
*  Entertainment :  7.883302296710117 %
*  Photo & Video :  4.9658597144630665 %
*  Educati

We may infer a couple of things about App Store and Google Play:

1. The most common genre for App Store is Games (a whooping 58.16%), followed by Entertainment (7.88%), Photo & Video (4.97%), Education (3.66%), Social Networking (3.29%), and others.

2. Notice that App Store common genres tend to be on the fun side (Games, Entertainment, etc) as opposed to on the practical side (Education, etc).

As for Google Play,

3. The most common genre or Google Play is Tools (8.45%), followed by Education (6.84%), Entertainment (6.42%), Business (4.59%), and Lifestyle (3.92%), and others.

4. The most common category of Google Play is FAMILY (18.93%), followed by GAME (9.69%), TOOLS (8.45%), BUSINESS (4.59%), and LIFESTYLE (3.90%), and others.

5. Google Play's genres/categories seems to be quite balanced between practical (Tools, Education, Business, Lifestyle) and fun aspects (Family, Game, Entertainment). This is in contrast to App Store that leans, particularly with the Games genre as higher than 50%.

Other observations are:

6. In all three frequency tables; the first, second, and third types "score" prominently higher than the bothers.

7. An app profile for App Store might be an engaging games (Games app), such as Minecraft.

8. An app profile for Google Play might be an app to help users with various or specific tasks (Tools app). It can also be an educative game targetted at children (Family-Game-Education app).

Notice that the frequency tables tell us about the most frequent genres/categories based on _the number of apps_, not on _the number of users_. It might be somewhat related, but this needs a further analysis.

### 2. Building App Profiles Based on Genres/Categories: An Alternative

Another way to find out the most popular genres is to calculate the average number of installs for each app genre, which can be done in Google Play by looking at the columns `'Installs'`. However, this information is missing in App Store dataset. As a compensation, we consider the total number of user ratings (the column `'rating_count_tot'`) for App Store.

We update the previous functions `freq_table()` and `display_table()` to cater this average based on column with the index of `added_index`:

In [16]:
def freq_table_2(dataset, index, added_index):
    result = {}
    for row in dataset:
        kws = row[index].split(";")
        for kw in kws:
            # We initially make result as a freq table with value as a list.
            # This list contains:
            # (1) accumulative row entries on (column w added_index)
            # (2) counts of the rows whose (column w index) entry equals key
            inc = float(row[added_index]) # In case if the entry is a string
            if kw in result:
                result[kw][0] += inc
                result[kw][1] += 1
            else:
                result[kw] = [inc, 1]
    # Convert result's values from list to an average
    for key in result:
        result[key] = result[key][0]/result[key][1]
    return result

def display_table_2(dataset, index, added_index, top_number = 10):
    # Similar to display_table but using freq_table_2
    for key, val in sorted(freq_table_2(dataset, index, added_index).items(), key = lambda x: x[1], reverse = True)[:top_number]:
        print("* ", key, ": ", val)

print("Ten most frequent prime_genre in App Store with average number of ratings:")
display_table_2(app_store, 11, 5) # col idx of reating_total_count is 5
print("\n")

Ten most frequent prime_genre in App Store with average number of ratings:
*  Navigation :  86090.33333333333
*  Reference :  74942.11111111111
*  Social Networking :  71548.34905660378
*  Music :  57326.530303030304
*  Weather :  52279.892857142855
*  Book :  39758.5
*  Food & Drink :  33333.92307692308
*  Finance :  31467.944444444445
*  Photo & Video :  28441.54375
*  Travel :  28243.8




Here, the most popular genre in App Store is Navigation, followed by Reference, Social Networking, Music, and Weather.

An app profie recommendation for App Store might be a map app that provides (1) user reviews to various places such as restaurant, and (2) weather forecasts in various areas. This covers Navigation, Reference, and Weather.

Back to Google Play, the `'Installs'` (col idx 5) numbers are apparently not precise enough. For example, "100+", "1,000+", "10,000,000+". To handle this, we delete the symbol "+" and "," from the entries of `'Installs'`, roughly converting the number. We update the `freq_table_2` to cater this string conversion, then output the frequency tables of Google Play based on Installs.

In [22]:
def freq_table_2(dataset, index, added_index):
    result = {}
    for row in dataset:
        kws = row[index].split(";")
        for kw in kws:
            # More detailed string convertion to number:
            # 1. If a string has "+" or ",", delete these using regex' sub()
            # 2. The type is still Class String. Convert to float
            from re import sub
            inc = float(sub("[\+,]", "", row[added_index]))
            if kw in result:
                result[kw][0] += inc
                result[kw][1] += 1
            else:
                result[kw] = [inc, 1]
    # Convert result value from list to an average
    for key in result:
        result[key] = result[key][0]/result[key][1]
    return result

print("Ten most frequent Genres in Google Play based on average install number:")
display_table_2(google_play, 9, 5)
print("\n")
print("Ten most frequent Category in Google Play based on average install number:")
display_table_2(google_play, 1, 5)
print("\n")

Ten most frequent Genres in Google Play based on average install number:
*  Communication :  38324327.08680555
*  Video Players & Editors :  24604573.25
*  Social :  23253652.127118643
*  Arcade :  21532908.755681816
*  Photography :  17805627.643678162
*  Productivity :  16787331.344927534
*  Casual :  16291911.818181818
*  Racing :  14744104.038461538
*  Travel & Local :  13984077.710144928
*  Action :  12390799.084507043


Ten most frequent Category in Google Play based on average install number:
*  COMMUNICATION :  38456119.167247385
*  VIDEO_PLAYERS :  24727872.452830188
*  SOCIAL :  23253652.127118643
*  PHOTOGRAPHY :  17805627.643678162
*  PRODUCTIVITY :  16787331.344927534
*  GAME :  15560965.599534342
*  TRAVEL_AND_LOCAL :  13984077.710144928
*  ENTERTAINMENT :  11640705.88235294
*  TOOLS :  10682301.033377837
*  NEWS_AND_MAGAZINES :  9549178.467741935




The most popular genre in Google Play based on average installs is Communication, followed by Video Players & Editors, and Social. Similarly, The most popular category in Google Play is Communication, followed by Video Players & Editors, and Social.

Based on this summary, a good app profile for Google Play might be an online messenger or a social media network, which covers both Communication and Social aspects. To cover Video Players & Editors aspect, this app can be designed with capability of sending, playing, and editing videos.

Eventually, our aim is to recommend an app genre that shows potential for being profitable on both Google Play and App Store. If we compare our initial app profiles of both platforms, there seems to be very little in common. However, the Communication/Social genre seems to be extremely high-scored in Google Play (rank 1 and 3) and quite high-scored in App Store (rank 3). Producing apps with this genre is also in line with our vaildation strategy: since we put our apps in Google Play first, it makes sense to prioritize the ranks in Google Play dataset. **Based on this, the recommended app genre would be Communication/Social.**

## Conclusion

We have done initializing the datasets of App Store and Google Play, exploring the datasets, data cleaning, and building an app profile for both targetted stores based on genres and categories. Our goal is to understand types of apps that possibly attract more users, using data analysis. We conclude that the recommended app genre/category for App Store and Google Play is Communication/Social.